In [ ]:
!pip3 install -r requirements.txt --break-system-packages

In [1]:
import sys
sys.path.insert(0, "~/.local/lib/python3.11/site-packages")

In [3]:
from transformers.utils.import_utils import is_nltk_available
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
%run mongo_query.ipynb

In [7]:
db
hebrew_channels = hebrew_channels

In [18]:
# find hebrew_channel with most messages
hebrew_channel_ids = list(hebrew_channels.keys())
cursor = db["dialogs"].find({"chat.id": {"$in": hebrew_channel_ids}})
top_channels = list(cursor)
# sort
top_channels.sort(key=lambda x: (x["top_message"]["id"]), reverse=True)

In [ ]:
top_channels_titles_and_counts = [
    (x["chat"]["id"], x["chat"]["title"], x["top_message"]["id"])
    for x in top_channels
]
print(top_channels_titles_and_counts)

In [23]:
abuali = -1001143765178

In [ ]:
db["messages"].find_one(
    {"chat.id": abuali},

)

In [41]:
result = db["messages"].find_one(
    {
        "chat.id": abuali,
        "$or": [
            {"text": {"$ne": None}},
            {"caption": {"$ne": None}},
            {"poll.question": {"$ne": None}},
        ],
    },
    {
        "from": "$chat.id",
        "date": 1,
        "text": {
            "$ifNull": [
                "$text",
                "$caption",
                "$poll.question",
            ]
        },
        "responses": "$poll.options",
        "reactions": "$reactions.reactions",
    }
)

In [ ]:
model.encode("hello")

In [ ]:
result

In [97]:
pipeline = [
    {
        "$match": {
            "chat.id": abuali,
            "$or": [
                {"text": {"$ne": None}},
                {"caption": {"$ne": None}},
            ],
            "reply_to_message_id": None,
        }
    },
    {
        "$project": {
            "cid": "$chat.id",
            "mid": "$id",
            "text": 1,
            "caption": 1,
            "reactions": 1,
            #"views": 1,
            "forwards": 1,
        }
    },
    {
        "$addFields": {
            "reactions_string": {
                "$reduce": {
                    "input": {
                        "$map": {
                            "input": "$reactions.reactions",
                            "as": "reaction",
                            "in": {
                                "$concat": [
                                    "$$reaction.emoji",
                                    ":",
                                    {"$toString": "$$reaction.count"},
                                    ", "
                                ]
                            }
                        }
                    },
                    "initialValue": "",
                    "in": {
                        "$concat": ["$$value", "$$this"]
                    }
                }
            }
        }
    },
    {
        "$addFields": {
            "text_with_reactions": {
                "$concat": [
                    "<<TEXT>>",
                    {"$replaceAll": {
                        "input": {"$ifNull": ["$text", "$caption"]},
                        "find": "\n\nכדי להגיב לכתבה לחצו כאן",
                        "replacement": "",
                    }},
                    "<<REACTIONS>>",
                    {"$rtrim": {"input": "$reactions_string", "chars": ", "}},
                    "<<FORWARDS>>",
                    {"$toString": "$forwards"},
                ]
            }
        }
    },
    {
        "$project": {
            "cid": 1,
            "mid": 1,
            "text": "$text_with_reactions",
        }
    },
    #{"$limit": 100}
]

results = db["messages"].aggregate(pipeline)

for result in results:
    cid, mid, text = result["cid"], result["mid"], result["text"]
    embed = model.encode(text)
    db["embeddings_2"].insert_one({
        "cid": cid,
        "mid": mid,
        "text": text,
        "embedding": embed.tolist(),
    })

# FIXME:
# this is an intermediate solution
# eventually we want:
# - a good encoder that we finetuned on this kind of data
# - a separate encoder for the sentiment (reactions & forwards)
# - and a model to unify them into a single representation
# also, there is no date, and this is terrible!
# date can be multidimensional because different "stories" have different meanings for different dates
# how do we fix this?